# The Problem
Putting a band class together, where each student selects 2 instruments, and a 3rd they'd be willing to play. 
We have an "ideal" band class composition, and want to get as close as possible to our ideal band class composition while still maintaining student happiness

### Constraints:

* Each student plays only 1 instrument.
* Each student cannot be assigned an instrument outside of their preferences.
* (optional) each instrument must be included in the final roster. 

### Optimize:

* minimize the difference between the ideal instrument count and the actual instrument count for each instrument.
* minimize the students who don't get their preferred instruments.


### Math

$$ 
let \\ N := \text{ number of students} \\
M := \text{ number of instruments} \\
\\
S_n := \text{ Student } n \quad \forall n \in \{1, 2, ..., N\} \\
P_n := \text{ Preferred instruments of student n} \\
\\
I_m := \text{ Instrument } m \quad \forall m \in \{1, 2, ..., M\} \\
C_m := \text{ Ideal count of instrument m} \\
\\
Y_{n,m} := \begin{cases}
    1, &\text{ if } P_n \text{ is assigned } I_m, \\
    0, &\text{ otherwise}
\end{cases}
$$


Therefore we have:
$$ Y = 
\begin{bmatrix} 
Y_{1,1} &Y_{1, 2} &\dots Y_{1,m}\\
Y_{2,1} &Y_{2, 2} &\dots Y_{1,m}\\
\vdots &\vdots &\ddots \vdots\\
Y_{n,1} &Y_{n, 2} &\dots Y_{n,m}\\
\end{bmatrix}
$$



Our Constraints

1. One Instrument

$\sum_{i=1}^m Y_{n, i} = 1 \quad \forall n \in \{1, 2, ..., N\}$

2. Student Selected Instruments

$Y_{n,m} = 0 \quad \forall I_m \not \in P_n$

3. Each Instrument Covered

$\sum_{i=1}^N Y_{m, i} >= 1 \quad \forall m \in \{1, 2, ..., M\}$

4. No instrument has more than twice it's ideal count of instruments.

$\sum_{i=1}^N Y_{m, i} <= 2C_m \quad \forall m \in \{1, 2, ..., M\}$


We seek to Optimize:

1. Class Composition

$MIN \quad \sum_{i=1}^m \bigg [ c_i - \sum_{j=1}^n Y_{i, j}\bigg]$

or even weight based on ideal count size (large sections are flexible), since we can't square the difference.

$MIN \quad \sum_{i=1}^m c_i \bigg [ c_i - \sum_{j=1}^n Y_{i, j}\bigg]$

2. Preferences

$MAX \quad \sum_{i=1}^n \bigg[\sum_{j \in P_i} Y _{i, j}\bigg]$

or even weight the preference based on position in preference list.

$MAX \quad \sum_{i=1}^n \bigg[\sum_{j \in P_i} j \cdot I(Y _{i, j} = P_i)\bigg]$
 

Therefore our function to minimize could be:

$MIN \quad \sum_{i=1}^m \bigg [ c_i - \sum_{j=1}^n Y_{i, j}\bigg] - \sum_{i=1}^n \bigg[\sum_{j \in P_i} Y _{i, j}\bigg]$


In [106]:
from time import time
from random import randint, sample
from collections import OrderedDict

import matplotlib.pyplot as plt
import pulp
import numpy as np
#https://github.com/ritvikmath/YouTubeVideoCode/blob/main/Shift%20Scheduling.ipynb

In [107]:
# Fake Data
instrument_idealcount = {
    "Flute" : 6,
    "Clarinet" : 6,
    "AltoSax" : 2,
    "TenorSax" : 2,
    "Trombone" : 4,
    "Baritone" : 3,
    "Trumpet" : 5,
    "Bass" : 1,
    "SnareDrum" : 1
    }
students_prefs = {f"Student{i}" : sample(list(instrument_idealcount.keys()), k=3) for i in range(30)}

# Setup
instruments = list(instrument_idealcount.keys())
counts = list(instrument_idealcount.values())
students = list(students_prefs.keys())
preferences = list(students_prefs.values())
num_instruments = len(instruments)
num_students = len(students)

COMPOSITION_WEIGHT = 2
PREFERENCE_WEIGHT = 5

students_prefs

{'Student0': ['Baritone', 'Flute', 'Trumpet'],
 'Student1': ['AltoSax', 'Flute', 'TenorSax'],
 'Student2': ['Baritone', 'Trumpet', 'TenorSax'],
 'Student3': ['Clarinet', 'SnareDrum', 'Bass'],
 'Student4': ['Bass', 'Trombone', 'Flute'],
 'Student5': ['Baritone', 'Bass', 'SnareDrum'],
 'Student6': ['Trombone', 'Trumpet', 'SnareDrum'],
 'Student7': ['Flute', 'Clarinet', 'AltoSax'],
 'Student8': ['AltoSax', 'SnareDrum', 'Bass'],
 'Student9': ['TenorSax', 'Baritone', 'Clarinet'],
 'Student10': ['Trumpet', 'Baritone', 'Flute'],
 'Student11': ['Bass', 'Flute', 'Clarinet'],
 'Student12': ['Clarinet', 'Trombone', 'Bass'],
 'Student13': ['AltoSax', 'Baritone', 'SnareDrum'],
 'Student14': ['SnareDrum', 'Baritone', 'Bass'],
 'Student15': ['Clarinet', 'AltoSax', 'TenorSax'],
 'Student16': ['SnareDrum', 'Trombone', 'Trumpet'],
 'Student17': ['Trombone', 'AltoSax', 'Baritone'],
 'Student18': ['Flute', 'Trumpet', 'SnareDrum'],
 'Student19': ['TenorSax', 'SnareDrum', 'AltoSax'],
 'Student20': ['Bariton

In [108]:
prob = pulp.LpProblem(name="BandClassOptimizer")

# Setup Matrix of Parameters
assignment = pulp.LpVariable.dicts(
    "Assignment", (students, instruments), lowBound = 0, upBound = 1, cat=pulp.LpInteger
    )

#Constraint 1: 1 instrument per person
for s in students:
    prob += (pulp.lpSum([assignment[s][i] for i in instruments]) == 1, f"{s}_will_play_1_instrument.")

# Constraint 2: Each instrument count > 0
for i in instruments:
    prob += (pulp.lpSum([assignment[s][i] for s in students]) >= 1, f"{i}_is_included.")

#Constraint 3: No non-selected instruments for each person.
for s in students:
    selected_instrument = students_prefs[s]
    non_selected_instruments = [instrument for instrument in instruments if instrument not in selected_instrument]
    prob += (pulp.lpSum([assignment[s][i] for i in non_selected_instruments]) == 0, f"{s}_instrument_selection")

# Constraint 4: Instrument count is less than or equal to twice it's ideal instrument count.
for i in instruments:
    prob += (pulp.lpSum([assignment[s][i] for s in students]) <= 2 * instrument_idealcount[i], f"Less_than_{2 * instrument_idealcount[i]}_{i}.")

#Objective Function:
obj = 0

# Ideal Band Composition:
#TODO: THIS HACK DOESN'T WORK AND GIVES BONUS FOR EXTRAS?? count - assigned_count is not always positive.
for i, count in instrument_idealcount.items():
    obj += (count - pulp.lpSum([assignment[s][i] for s in students])) * count * COMPOSITION_WEIGHT

# Student Preferred Instruments:
for s, preferences in students_prefs.items():
    obj += pulp.lpSum([assignment[s][i] * (preferences.index(i)) for i in preferences]) * PREFERENCE_WEIGHT

prob += obj


In [109]:
start = time()
status = prob.solve()
end = time()
seconds = end - start
print('Status:', 'success' if status==1 else 'failure: 0 < section size < 2 * (ideal section size) impossible with current student preferences.')
print('Seconds:' if seconds > 1 else 'Milliseconds:', round(seconds,2) if seconds > 1 else int(seconds*1000))

output_assignments = np.array([int(v.varValue) for v in prob.variables()]).reshape(num_students, num_instruments)
# if status == 1:

#     plt.figure()
#     plt.figure(figsize=(num_students*1.5, num_instruments*1.5))
#     im = plt.imshow(output_assignments, aspect='equal', cmap='gray')
#     ax = plt.gca()

#     ax.set_xticks(np.arange(0, num_instruments, 1))
#     ax.set_yticks(np.arange(0, num_students, 1))

#     # Labels for major ticks
#     ax.set_xticklabels(sorted(instruments), fontsize=16)
#     ax.set_yticklabels(sorted(students), fontsize=16)
#     plt.xticks(rotation = 90)

#     # Minor ticks
#     ax.set_xticks(np.arange(-.5, num_instruments, 1), minor=True)
#     ax.set_yticks(np.arange(-.5, num_students, 1), minor=True)

#     # Gridlines based on minor ticks
#     ax.grid(which='minor', color='gray', linestyle='-', linewidth=2)

#     # Title and Formatting
#     num_prefs_matched = int(prob.objective.value())
#     plt.suptitle(f'{num_prefs_matched} off of perfect band composition.', fontsize=25)
#     plt.title("Based on ideal instrument counts and student preferences.")

#     plt.show()



Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Mar 14 2022 

command line - cbc /var/folders/15/05knpnr51lj1sbhrslpf21rc0000gp/T/1339e7f9eb2d4a9b8411bbb4e48ae0d8-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/15/05knpnr51lj1sbhrslpf21rc0000gp/T/1339e7f9eb2d4a9b8411bbb4e48ae0d8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 83 COLUMNS
At line 1884 RHS
At line 1963 BOUNDS
At line 2234 ENDATA
Problem MODEL has 78 rows, 270 columns and 990 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -206 - 0.00 seconds
Cgl0002I 180 variables fixed
Cgl0004I processed model has 39 rows, 90 columns (90 integer (90 of which binary)) and 180 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -206
Cbc0038I Before mini branch and bound, 90 integers at bound fixed and 0 con

In [110]:
import pandas as pd 
import altair as alt

# Replace assign with preference.
# vars = prob.variables()
# vars.sort(key= lambda v: int(str(v).split("_")[1].replace("Student", "")))
# assignment_prefs = np.array(list(map(lambda var: int(var.varValue), vars))).reshape(num_students, num_instruments)

# for s, student in enumerate(sorted(students)):
#     for i, instrument in enumerate(sorted(instruments)):
#         if assignment_prefs[s][i] and instrument in students_prefs[student]:
#             assignment_prefs[s][i] = students_prefs[student].index(instrument) + 1

# # Pre-sort student list to match LpVariables alphabetical
# df = (
#     pd.DataFrame(assignment_prefs, columns = sorted(instruments))
#     .set_index(pd.Series(sorted(students)))
# )

In [111]:
df = pd.DataFrame(assignment).applymap(pulp.value).transpose()
df = df.melt(var_name="instrument", ignore_index=False, value_name="assignment").reset_index().rename(columns={"index" : "student"})
df["preference"] = df.apply(
    lambda row: 
        students_prefs[row.student].index(row.instrument) + 1 # +1 to offset indexing 
        if row.assignment else 0, 
    axis = 1)
print(df)
basechart = (
    alt.Chart(df)
    .encode(x=alt.X("instrument", sort=instruments, title=None), 
            y=alt.Y("student", sort= students, title=None),
        )
    .properties(
            title = {
                "text": "Optimal Band Assignment.",
                "subtitle" : "Student Instrument Assignments and Preferences.",
                "fontSize": 20,
                "anchor": "start"
                }
        )
    )
chart_rectangles = (
    basechart.mark_rect(stroke="black", strokeWidth=0.1)
    .encode(color=alt.Color("preference:O", scale=alt.Scale(scheme="redgrey", reverse=True), title="Instrument Preference"))
)

chart_text = (
    basechart.mark_text(color="grey")
    .encode(text="preference:O")
    .transform_filter(alt.datum.preference > 0)
)

chart = chart_rectangles + chart_text
chart

       student instrument  assignment  preference
0     Student0      Flute         1.0           2
1     Student1      Flute         1.0           2
2     Student2      Flute         0.0           0
3     Student3      Flute         0.0           0
4     Student4      Flute         0.0           0
..         ...        ...         ...         ...
265  Student25  SnareDrum         0.0           0
266  Student26  SnareDrum         0.0           0
267  Student27  SnareDrum         0.0           0
268  Student28  SnareDrum         0.0           0
269  Student29  SnareDrum         1.0           1

[270 rows x 4 columns]


alt.LayerChart(...)

In [112]:
# Display Band Composition.
instrument_counts = {instrument : 0 for instrument in instruments}
instrument_assignments = {instrument : [] for instrument in instruments}
for s in students:
    for i in instruments:
        if pulp.value(assignment[s][i]) == 1:
            instrument_counts[i] += 1
            instrument_assignments[i].append(s)
            
print(f"-=============== ASSIGMENTS BY SECTION ===============-")
for instrument, count in instrument_counts.items():
    print(f"{instrument.upper()}: [ {count} / {instrument_idealcount[instrument]} ]")
    print(f"\t-{instrument_assignments[instrument]}")
    # for student in instrument_assignments[instrument]:
    #     print(f"\t- {student}")


-=============== ASSIGMENTS BY SECTION ===============-
FLUTE: [ 7 / 6 ]
	-['Student0', 'Student1', 'Student7', 'Student11', 'Student18', 'Student21', 'Student27']
CLARINET: [ 6 / 6 ]
	-['Student3', 'Student12', 'Student15', 'Student20', 'Student25', 'Student26']
ALTOSAX: [ 2 / 2 ]
	-['Student8', 'Student13']
TENORSAX: [ 2 / 2 ]
	-['Student9', 'Student19']
TROMBONE: [ 5 / 4 ]
	-['Student4', 'Student6', 'Student16', 'Student17', 'Student22']
BARITONE: [ 3 / 3 ]
	-['Student2', 'Student5', 'Student23']
TRUMPET: [ 2 / 5 ]
	-['Student10', 'Student24']
BASS: [ 1 / 1 ]
	-['Student28']
SNAREDRUM: [ 2 / 1 ]
	-['Student14', 'Student29']
